# Run astrometry.net
Try to run astrometry.net on an image from a notebook.

In [ ]:
import os
from du_astro_utils import calibration, photometry, utils
from astropy.io import fits
from astropy.table import Table
import sys

In [ ]:
target_type = "exoplanet"
if target_type == "asteroid":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_ASTER)
elif target_type == "exoplanet":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_EXPLTS)
os.listdir(data_dir)

In [ ]:
for ddir in os.listdir(data_dir):
    subdata_dir = os.path.join(data_dir, ddir)
    if os.path.isdir(subdata_dir):
        list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
        list_fits = sorted(list_fits)
        print(subdata_dir, len(list_fits))

In [ ]:
achoice = os.listdir(data_dir)[0]
aligned = False
subdata_dir = os.path.join(data_dir, achoice)
if aligned:
    subdata_dir = os.path.join(subdata_dir, "aligned")
list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
list_fits = sorted(list_fits)
list_fits

In [ ]:
loc = 0
scimage = list_fits[loc]
fits_sci_image = os.path.join(subdata_dir, scimage)
fits_sci_image = os.path.abspath(fits_sci_image)
if os.path.isfile(fits_sci_image):
    # Get image directory, failename and extension
    sc_im_dir = os.path.abspath(os.path.dirname(fits_sci_image))
    sc_im_name, sc_im_ext = os.path.splitext(os.path.basename(fits_sci_image))

    bias_dir, darks_dir, flats_dir = utils.get_calib_dirs_photometry(fits_sci_image)

    # Get information from FITS header
    sc_date, sc_scope, sc_cam, sc_filter, sc_expos, sc_x, sc_y = calibration.get_infos_from_image(fits_sci_image, verbose=False)

    # Run calibration
    dico_calib = calibration.reduce_sci_image(fits_sci_image, darks_dir, flats_dir, path_to_bias_dir="", use_bias=False, override_date_check=True, max_days=7, speedup=True, verbose=True)

    # Analysis
    red_sci_image = dico_calib["path"]
red_sci_image

In [ ]:
with fits.open(red_sci_image) as hdul:
    hdu = hdul[0]
    print(hdu.header)
    hdu_data = hdu.data

In [ ]:
from astropy.stats import sigma_clipped_stats

mean, med, sig = sigma_clipped_stats(hdu_data)
mean, med, sig

In [ ]:
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style

plt.imshow(hdu_data, vmin=med - 5 * sig, vmax=med + 5 * sig)
plt.colorbar()

In [ ]:
im_dir = os.path.abspath(os.path.dirname(red_sci_image))
im_name, im_ext = os.path.splitext(os.path.basename(red_sci_image))

In [ ]:
import numpy as np

new_data = np.flip(hdu_data, axis=1)
with fits.open(red_sci_image) as hdul:
    hdu = hdul[0]
    hdu.data = new_data
    transp_im = os.path.join(im_dir, f"{im_name}_transpose{im_ext}")
    hdu.writeto(transp_im)

In [ ]:
out_dir = os.path.join(im_dir, "solved_astrometry")

solve_cmd = f"solve-field --batch --out tmp_solve --downsample 2 --no-plots -D {out_dir} {transp_im} --overwrite"
solve_cmd

In [ ]:
os.system(solve_cmd)

In [ ]:
os.system("ls ${ARCHIVESC2PU}/index5200_astrometry")

In [ ]:
cat_tab = Table.read(os.path.join(out_dir, f"{im_name}.new"))

In [ ]:
cat_tab

In [ ]:
fwhm_est

SEXTRACTOR has been run and compatibility ensured with the fwhm function in the photometry module.